Use the cells with the Create Hits, Retrieve Results, and Process Results functions.


In [3]:
import boto3
import xmltodict
import json, os, csv
import pandas as pd
from generate_batch import make_batch

# Change this to True to publish hits outside the sandbox
create_hits_in_production = False
environments = {
  "production": {
    "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
    "preview": "https://www.mturk.com/mturk/preview"
  },
  "sandbox": {
    "endpoint": 
          "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
    "preview": "https://workersandbox.mturk.com/mturk/preview"
  },
}
mturk_environment = environments["production"] if create_hits_in_production else environments["sandbox"]
session = boto3.Session(profile_name='mturk')
client = session.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
)


In [4]:
#########################
##Files and Directories##
#########################

#name of experiment - each experiment has its own seperate annotations and hits
EXPERIMENT = "Full_Annotations"

#Retrieved results go here - unprocessed
RAW_ANNOTATIONS_CSV = f"experiments/{EXPERIMENT}/raw_annotations.csv" 

HTML_FILE_DRAWN = 'task.html'

#processed results here - each pair shows up once, and a count for the number of times each tile was chosen as most legible
ANNOTATIONS_CSV = f"experiments/{EXPERIMENT}/annotations.csv" 

#path to available hits csv, where we keep all currently published hits for record
HITS_CSV = f'experiments/{EXPERIMENT}/hits.csv'

#The directory where we store the pairs of tiles in CSV format that will be used to create hits
INPUT_DIREC = f'experiments/{EXPERIMENT}/hits'   

#The tiles our hits are sampling from
TILES_LIST = 'point4-1000-pairs-fillable.txt'

#CSV holding optional feedback answers
FEEDBACK_CSV = 'feedback.csv'

#Source of images on Scott's website
IMGS_SRC = 'https://facultyweb.cs.wwu.edu/~wehrwes/borders/1k/' 
#IMGS_SRC = 'https://facultyweb.cs.wwu.edu/~wehrwes/borders/pilot100/' 

if not os.path.exists(f"experiments/{EXPERIMENT}"):
    os.mkdir(f"experiments/{EXPERIMENT}")
if not os.path.exists(INPUT_DIREC):
    os.mkdir(INPUT_DIREC)

In [5]:
def generate_qualification_test(name, keywords, description, duration):
    questions = open('qual_test.xml', mode='r').read()   
    answers = open('qual_answers.xml', mode='r').read()

    qual_response = client.create_qualification_type(
                            Name=name,
                            Keywords=keywords,
                            Description=description,
                            QualificationTypeStatus='Active',
                            Test=questions,
                            AnswerKey=answers,
                            TestDurationInSeconds=duration * 60)
    print(qual_response['QualificationType']['QualificationTypeId'])
    
#html_file - the html for the task
def create_hits(num_hits, num_assignments, task_attributes, html_file, num_pairs=30):
    csv_file = make_batch(TILES_LIST, INPUT_DIREC, task_attributes['Title'], num_hits, IMGS_SRC)
    df = pd.read_csv(csv_file)
    with open(HITS_CSV, "a+") as f:
        writer = csv.writer(f)
        if os.stat(HITS_CSV).st_size == 0:    
            writer.writerow(["batch", "hit_id"])    
        QUESTION_XML = """<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">
            <HTMLContent><![CDATA[{}]]></HTMLContent>
            <FrameHeight>650</FrameHeight>
            </HTMLQuestion>"""
        results = []
        for i in range(df.shape[0]):
            html_layout = open(html_file, 'r').read()
            j = 0
            for image in df.columns:  
                html_layout = html_layout.replace('${' + image + '}', df.iat[i, j] + '.jpeg')
                html_layout = html_layout.replace(image + '_drawn', df.iat[i, j] + '_drawn.jpeg')
                html_layout = html_layout.replace(image, df.iat[i, j] + '.jpeg')           
                j += 1        
            question_xml = QUESTION_XML.format(html_layout)
            response = client.create_hit(**task_attributes, Question=question_xml)
            hit_type_id = response['HIT']['HITTypeId']
            row = [task_attributes['Title'], response['HIT']['HITId']]
            writer.writerow(row)
    print("You can view the HITs here:")
    print(mturk_environment['preview']+"?groupId={}".format(hit_type_id))

        

In [38]:
def retrieve_results():
    #which columns our counts are in, in case we want to add additional columns in the future
    HIT_ID_COL = 1
    
    NUM_PAIRS = 30
    #df is a list of HITs currently in circulation
    df = pd.read_csv(HITS_CSV)
    #loop through every hit
    with open(RAW_ANNOTATIONS_CSV, "a+") as f:
        writer = csv.writer(f)
        if os.stat(RAW_ANNOTATIONS_CSV).st_size == 0: 
            writer.writerow(["tile1","tile2","choice","worker_id","batch_name","hit_id","assignment_id","submit_time"])
        for h in range(df.shape[0]):
            hit_id = df.iat[h, HIT_ID_COL]
            hit = client.get_hit(HITId=hit_id)
            #get all assignments for the hit that have been submitted
            assignmentsList = client.list_assignments_for_hit(
                    HITId=hit_id,
                    AssignmentStatuses=['Submitted']
                )
            assignments = assignmentsList['Assignments']
            for assignment in assignments:
                answer_dict = xmltodict.parse(assignment['Answer'])
                assignment_id = assignment['AssignmentId']
                worker_id = assignment['WorkerId']
                submit_time = assignment['SubmitTime']
                batch_name = hit['HIT']['Title']
                #if feedback was given, then the first answer is feedback, and we have to account for that
                offset = 1 if len(answer_dict['QuestionFormAnswers']['Answer']) > NUM_PAIRS * 2 else 0
                #process every pair in the assignment
                for i in range(0, NUM_PAIRS):
                    answer1 = answer_dict['QuestionFormAnswers']['Answer'][2 * i + offset]['FreeText'] == "true"          
                    tile1  = answer_dict['QuestionFormAnswers']['Answer'][2* i + offset]['QuestionIdentifier'][-18:-5]
                    tile2  = answer_dict['QuestionFormAnswers']['Answer'][2* i + 1 + offset]['QuestionIdentifier'][-18:-5]
                    choice = tile1 if answer1 else tile2
                    row = [tile1, tile2, choice, worker_id,batch_name,hit_id,assignment_id,submit_time]
                    writer.writerow(row)

                #if there is feedback
                if offset == 1:
                    try:
                        df_feedback = pd.read_csv(FEEDBACK_CSV)
                    except:
                        df_feedback = pd.DataFrame()
                    feedback = answer_dict['QuestionFormAnswers']['Answer'][0]['FreeText']
                    row = pd.DataFrame([{'worker_id': worker_id, 'batch_name':batch_name, 'feedback':feedback}])
                    df_feedback = pd.concat([df_feedback, row])
                    df_feedback.to_csv(FEEDBACK_CSV)
                #approve assignment
                if assignment['AssignmentStatus'] == 'Submitted':
                            client.approve_assignment(
                            AssignmentId=assignment_id,
                            OverrideRejection=False
                        )  
            #delete the hit if every assignment has been submitted
            delete_hit(hit_id)      

# Note: this function is a little buggy, mostly due to deleting HITs thru the API being spotty
def delete_hit(hit_id):
    df = pd.read_csv(HITS_CSV)
    #print(df)
    check = False
    try:   
        row = df.loc[(df['hit_id'] == hit_id)].index[0]
    except:
        return
    try:
        client.delete_hit(HITId=hit_id)
        check = True
    except:
        print("RequestError occured, couldn't delete hit")
    if check or client.get_hit(HITId = hit_id)['HIT']['HITStatus'] == 'Disposed':
        df = df.drop(df.index[row])
    df.to_csv(HITS_CSV, index=False)

In [39]:
#################
##Creating HITs##
#################

#Run this block to create and upload HITs to Mturk

# Below are configurations that can be changed
###############################################################################################
# Number of unique hits to be created
num_hits = 10

# Number of workers you want completing each hit
num_assignments = 1

# The name of the hit that will be shown to workers. We use this name to keep track of data as well
# Recommend putting the date of which the batch was published in the name
batch_name = 'Test'

# ID of the qualification test needed to take this hit. If you generate a new qual test, or want workers to take it again,
# be sure you change this with the updated ID
qualification_id = "33ADZZWSDZ1YR8VF94TSXDAB2378WR"

# Number of days hit is available for
num_days = 28

# Duration workers have to finish task once started (in minutes)
task_duration = 60

# How much each worker is compensated for per hit (for some reason this is a string)
reward = '1.00'

# Keywords - descriptive words that a worker may use to find our task
keywords = 'borders, satellite, political-science, choose'

# Description of task
description = 'Given a pair of images of borders, choose which border is more legible'

###############################################################################################
# don't change this
task_attributes = {
    # Number of assignments per HIT
    'MaxAssignments': num_assignments,           
    # How long the task will be available on MTurk     
    'LifetimeInSeconds': 60*60*24*num_days,
    # How long Workers have to complete each item
    'AssignmentDurationInSeconds': 60*task_duration,
    # The reward you will offer Workers for each response
    'Reward': reward,
    'Title': batch_name,
    'Keywords': keywords,
    'Description': description,
    'QualificationRequirements': [{'QualificationTypeId': qualification_id,
                                   'Comparator': 'EqualTo',
                                   'IntegerValues':[100]}, 
                                 {'QualificationTypeId' : "00000000000000000071",
                                 'Comparator': 'EqualTo',
                                   'LocaleValues':[{'Country':'US'}]}]
    
    } 
create_hits(num_hits, num_assignments,task_attributes, HTML_FILE_DRAWN)

You can view the HITs here:
https://workersandbox.mturk.com/mturk/preview?groupId=36FWM6US0VQV4TDGFBHCRHZDOTZS4H


In [9]:
######################
##Qualification Test##
######################

# Creating a new test returns a Qualification ID. Make sure to attach this to any new hits created in order to have
# a worker take this test

name = "Test"
keywords = 'test, qualification, borders, images'
description = 'A brief test to see if you understand what a legible border is'
test_duration = 10 # in minutes
generate_qualification_test(name, keywords, description, test_duration)


33ADZZWSDZ1YR8VF94TSXDAB2378WR


In [42]:
######################
##Retrieving Results##
######################

#Run this block to fetch results for our hits
#It goes through all hits in circulation (what appears in hits.csv) and fetches results for those hits
#Results are stored in raw_annotations.csv, which tracks: the pair, selected, worder_id, hit_id, assignment_id, time submitted
#Then deletes hits that have all assignments completed

retrieve_results()

RequestError occured, couldn't delete hit
RequestError occured, couldn't delete hit
RequestError occured, couldn't delete hit
RequestError occured, couldn't delete hit
RequestError occured, couldn't delete hit
RequestError occured, couldn't delete hit
RequestError occured, couldn't delete hit
RequestError occured, couldn't delete hit


In [5]:

#takes annotations and fills a csv file with pairs and counts for each tile
def process_results(blacklist=[], csv_file=RAW_ANNOTATIONS_CSV, output=ANNOTATIONS_CSV):
    try:
        raw_df = pd.read_csv(csv_file)
    except:
        print("error: no file found for: {}".format(csv))
        return
    with open(output, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["tile1", "tile2", "tile1count", "tile2count"])
    df = pd.read_csv(output)
    for i in range(raw_df.shape[0]):
        if raw_df["worker_id"][i] in blacklist:
            continue
        tile1 = min(raw_df.iat[i,0],raw_df.iat[i,1])
        tile2 = max(raw_df.iat[i,0],raw_df.iat[i,1])
        selection = raw_df.iat[i,2]
        choice = [1,0] if selection == tile1 else [0,1]
        if df.loc[((df['tile1'] == tile1) & (df['tile2'] == tile2))].empty:
            row = pd.Series([tile1, tile2, choice[0], choice[1]], index=df.columns)
            df = df.append(row, ignore_index=True)
        else:
            index = df.loc[((df['tile1'] == tile1) & (df['tile2'] == tile2))].index[0]
            df.iat[index, 2] += choice[0]
            df.iat[index, 3] += choice[1]
    df.to_csv(output, index=False)

In [8]:
# functions to visualize the collected data

## helpers

def imgid_to_imgtag(image_id, drawn=True):
    country_pair = "-".join(image_id.split("-")[:2])
    base = f"{IMGS_SRC}global-rectified-point4/{country_pair}/{image_id}/{image_id}"
    url = f"{base}_drawn.jpeg" if drawn else f"{base}.jpeg"
    return f'<img src="{url}"/>'

def table_row(df_row):
    
    return f"""<tr>
        <td>{imgid_to_imgtag(df_row['tile1'])}<br/>{df_row['tile1count']}</td>
        <td>{imgid_to_imgtag(df_row['tile2'])}<br/>{df_row['tile2count']}</td>
        </tr>"""
  
# visualize aggregated pairs with vote counts below each tile
def visualize_results(csv=ANNOTATIONS_CSV):
    try:
        df = pd.read_csv(csv)
    except FileNotFoundError as e:
        print(f"annotations file {ANNOTATIONS_CSV} could not be read")
        return

    df.apply(table_row, axis=1)
    content = "\n".join(df.apply(table_row, axis=1).tolist())
    output = f"""
    <html>
    <head><title>{"Results_Full"}</title></head>
    <center>
    <table>
    {content}
    </table></center></html>
    """
    
    with open(f"experiments/{EXPERIMENT}/results_aggregated.html", 'w+') as f:
        f.write(output)
            

def border_div(tile):
    return f'<div style="border: 4px solid;">{tile}</div>'

def noborder_div(tile):
    return f'<div style="border: 4px solid white;">{tile}</div>'

# visualize raw results with a table per annotator
def visualize_raw_results(csv=RAW_ANNOTATIONS_CSV):
    try:
        raw_df = pd.read_csv(csv)
    except:
        print("error: no file found for: {}".format(csv))
        return

    t1chosen = raw_df["tile1"] == raw_df["choice"]
    t2chosen = raw_df["tile2"] == raw_df["choice"]
    
    raw_df["tile1"] = raw_df["tile1"].apply(imgid_to_imgtag)
    raw_df["tile2"] = raw_df["tile2"].apply(imgid_to_imgtag)

    raw_df["tile1"][t1chosen] = raw_df["tile1"][t1chosen].apply(border_div)
    raw_df["tile1"][~t1chosen] = raw_df["tile1"][~t1chosen].apply(noborder_div)

    raw_df["tile2"][t2chosen] = raw_df["tile2"][t2chosen].apply(border_div)
    raw_df["tile2"][~t2chosen] = raw_df["tile2"][~t2chosen].apply(noborder_div)
    
    with open("results_raw_full.html", "w") as f:
        annotators = raw_df.groupby("worker_id")
        for worker_id, df in annotators:
            f.write(f"<h2>{worker_id}</h2>")
            df[["tile1", "tile2", "worker_id"]].to_html(buf=f, escape=False, border=0, index=False)
  
'''
Takes in a csv with with the column 'name' and turns the image names
into a HTML file in rows of 5. Images are shown in the order listed 
by the csv
'''
def visualize_results_rank(csv=ANNOTATIONS_CSV):
    try:
        df = pd.read_csv(csv)
    except FileNotFoundError as e:
        print(f"annotations file {ANNOTATIONS_CSV} could not be read")
        return
    
    N = len(df)
    content = ""
    for i in range(0, N, 5):
        block = '\n'.join('<td>' + 
                          imgid_to_imgtag(df['name'][i+j]) + 
                          '</td>' for j in range(5) if (i+j) < N
                        )
        content += f"""<tr>
            {block}
        </tr>"""
        content += "\n"
    output = f"""
    <html>
    <head><title>{'Rank Results'}</title></head>
    <center>
    <table>
    {content}
    </table></center></html>
    """
    
    with open("results_rank_full.html", 'w') as f:
        f.write(output)
            


visualize_results_rank()


In [6]:
###################
##Process Results##
###################

# Run this to add all collected annotations to our list of processed pairs.
# This stores data in annotations.csv. The file tracks each pair only once, and the counts for each image
# Running this function does NOT delete anything from the raw_annotations. This will overwrite the contents of annotations.csv

# add any worker ID's to this that you do not want to process
blacklist = []
process_results(blacklist)

/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

...

/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_1998/726852067.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [41]:
# generate html visualizations from ANNOTATIONS_CSV and RAW_ANNOTATIONS_CSV
batch_name = 'Border Legibility Pilot'
visualize_results()
#visualize_raw_results()